In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq

llm=ChatGroq(model="qwen-2.5-32b")

In [3]:
def add(a:int,b:int)-> int:
    """Add a and b.

    Args:
        a: first int
        b: second int
    """
    return a+b

In [4]:
llm_with_tools=llm.bind_tools([add])

In [5]:
tool_call=llm_with_tools.invoke("What is 2 plus 3")

In [6]:
tool_call

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5689', 'function': {'arguments': '{"a": 2, "b": 3}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 192, 'total_tokens': 217, 'completion_time': 0.125, 'prompt_time': 0.012240433, 'queue_time': 0.24785531, 'total_time': 0.137240433}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_35f92f8282', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d2b980b0-1e79-4423-be9c-76a94425fb4e-0', tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 3}, 'id': 'call_5689', 'type': 'tool_call'}], usage_metadata={'input_tokens': 192, 'output_tokens': 25, 'total_tokens': 217})

In [7]:
tool_call=llm_with_tools.invoke("What is GenAI?")

In [8]:
tool_call

AIMessage(content='GenAI, short for Generative Artificial Intelligence, refers to a category of AI models that are designed to create new, original content such as text, images, audio, or video. These models are typically trained on large datasets and use techniques like deep learning to generate outputs that are similar to the training data. Examples of GenAI include language models like GPT (Generative Pre-trained Transformer), which can write essays, stories, or even computer code, and image-generating models like DALL-E or Stable Diffusion, which can create digital images or art pieces based on textual descriptions.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 119, 'prompt_tokens': 190, 'total_tokens': 309, 'completion_time': 0.595, 'prompt_time': 0.011121235, 'queue_time': 0.247935348, 'total_time': 0.606121235}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_35f92f8282', 'finish_reason': 'stop', 'logprobs': None}, id='run-94dbbd60-e764-

In [9]:
tool_call=llm_with_tools.invoke("What is 2 multiply by 3")

In [10]:
tool_call

AIMessage(content='The question seems to be asking for a multiplication operation, but the provided tool only supports addition. However, I can directly answer your question without using the tool. \n\n2 multiplied by 3 is 6.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 193, 'total_tokens': 236, 'completion_time': 0.215, 'prompt_time': 0.012021191, 'queue_time': 0.28030073, 'total_time': 0.227021191}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_35f92f8282', 'finish_reason': 'stop', 'logprobs': None}, id='run-081e6bbd-38cd-42e2-aee9-18d135391a95-0', usage_metadata={'input_tokens': 193, 'output_tokens': 43, 'total_tokens': 236})

In [ ]:
# 📌 Import Required Libraries
import os
from dotenv import load_dotenv
from typing import Annotated
from typing_extensions import TypedDict

from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage, AnyMessage  # ✅ Fix import

# ✅ Load Environment Variables
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

# ✅ Define Chat State using TypedDict with `add_messages`
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]  # Stores all chat messages

# ✅ Initialize LangGraph Execution Model
graph_builder = StateGraph(State)

# ✅ Load Groq LLM Model (gemma2-9b-it)
llm = ChatGroq(model="gemma2-9b-it")

# ✅ Define the Chatbot Function
def chatbot(state: State):
    """
    Processes user input and returns AI-generated messages.
    """
    user_input = state["messages"][-1].content  # ✅ Extract latest user message
    response = llm.invoke(user_input)  # ✅ Send only the latest message
    return {"messages": [AIMessage(content=response.content)]}  # ✅ Convert to AIMessage

# ✅ Build the AI Graph
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")  # Start -> chatbot
graph_builder.add_edge("chatbot", END)  # chatbot -> End
graph = graph_builder.compile()

# ✅ Function to Stream AI Responses and Retrieve Full Conversation
def stream_graph_updates(user_input: str):
    """
    Streams AI-generated responses and prints all messages dynamically.
    """
    for event in graph.stream({"messages": [HumanMessage(content=user_input)]}):  # ✅ Use HumanMessage
        for value in event.values():
            # ✅ Get all messages tracked by LangGraph
            all_messages = value["messages"]

            # ✅ Print full conversation
            print("\n--- Full Conversation ---")
            for msg in all_messages:
                role = "User" if isinstance(msg, HumanMessage) else "Assistant"
                print(f"{role}: {msg.content}")

    for message in state['messages']:
        print(message)

# ✅ Start Chatbot Interaction
def run_chatbot():
    """
    Runs the chatbot in an interactive terminal loop.
    """
    print("\n🤖 LangGraph Chatbot (Type 'exit' to quit)\n")
    while True:
        try:
            user_input = input("User: ")
            if user_input.lower() in ["quit", "exit", "q"]:
                print("Goodbye! 👋")
                break

            stream_graph_updates(user_input)
        except KeyboardInterrupt:
            print("\nSession ended. Goodbye! 👋")
            break
        except Exception as e:
            print(f"Error: {e}")

# ✅ Run the Chatbot
if __name__ == "__main__":
    run_chatbot()



🤖 LangGraph Chatbot (Type 'exit' to quit)


--- Full Conversation ---
Assistant: Hi! 👋  How can I help you today? 😊


--- Full Conversation ---
Assistant: **Generative AI (Gen AI)** is a type of artificial intelligence that focuses on creating new content, rather than simply analyzing or understanding existing data. 

Here's a breakdown:

**What it does:**

* **Generates:**  It produces new outputs like text, images, audio, video, code, 3D models, and more. 
* **Learns from Data:** It's trained on massive datasets of existing content, learning patterns and structures within that data.
* **Creates Novel Outputs:** Using its learned knowledge, it can generate original content that is similar in style and substance to the training data, but not simply a copy.

**Examples:**

* **Text Generation:** ChatGPT (like me!), Bard, Jasper, writing assistants
* **Image Generation:** DALL-E 2, Stable Diffusion, Midjourney (creating realistic or artistic images from text prompts)
* **Audio Generatio

In [ ]:
def display(state):
    for message in state['messages']:
        print(message)



TypeError: 'CompiledStateGraph' object is not subscriptable

In [ ]:
# 📌 Import Required Libraries
import os
from dotenv import load_dotenv
from typing import Annotated
from typing_extensions import TypedDict

from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage, AnyMessage

# ✅ Load Environment Variables
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

# ✅ Define Chat State using TypedDict with `add_messages`
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]  # Stores all chat messages

# ✅ Initialize LangGraph Execution Model
graph_builder = StateGraph(State)

# ✅ Load Groq LLM Model (gemma2-9b-it)
llm = ChatGroq(model="gemma2-9b-it")

# ✅ Define the Chatbot Function
def chatbot(state: State):
    """
    Processes user input and returns AI-generated messages.
    """
    user_input = state["messages"][-1].content  # Extract latest user message
    response = llm.invoke(user_input)  # Get AI response
    return {"messages": [AIMessage(content=response.content)]}  # Convert to AIMessage


# ✅ Define the Function to Display the Full Conversation
def display(state: State):
    """
    Displays the entire conversation history.
    """
    print("\n--- Full Conversation ---")
    for message in state["messages"]:
        role = "User" if isinstance(message, HumanMessage) else "Assistant"
        print(f"{role}: {message.content}")
    return state  # Return state to continue execution


# ✅ Build the AI Graph
graph_builder.add_node("chatbot", chatbot)  # AI response node
graph_builder.add_node("display", display)  # Display node

# ✅ Define Graph Execution Order
graph_builder.add_edge(START, "chatbot")  # Start -> chatbot
graph_builder.add_edge("chatbot", "display")  # chatbot -> display
graph_builder.add_edge("display", END)  # display -> End

# ✅ Compile the Graph
graph = graph_builder.compile()

def stream_graph_updates(user_input: str):
    """
    Streams AI-generated responses and prints all messages dynamically.
    """
    
    for event in graph.stream({"messages": [HumanMessage(content=user_input)]}):  # ✅ Use HumanMessage
        for value in event.values():
            # ✅ Get all messages tracked by LangGraph
            all_messages = value["messages"]

            # ✅ Print full conversation
            print("\n--- Full Conversation ---")
            for msg in all_messages:
                role = "User" if isinstance(msg, HumanMessage) else "Assistant"
                print(f"{role}: {msg.content}")


# ✅ Start Chatbot Interaction
def run_chatbot():
    """
    Runs the chatbot in an interactive terminal loop.
    """
    print("\n🤖 LangGraph Chatbot (Type 'exit' to quit)\n")
    while True:
        try:
            user_input = input("User: ")
            if user_input.lower() in ["quit", "exit", "q"]:
                print("Goodbye! 👋")
                break

            stream_graph_updates(user_input)
        except KeyboardInterrupt:
            print("\nSession ended. Goodbye! 👋")
            break
        except Exception as e:
            print(f"Error: {e}")

# ✅ Run the Chatbot
if __name__ == "__main__":
    run_chatbot()



🤖 LangGraph Chatbot (Type 'exit' to quit)


--- Full Conversation ---
Assistant: Hello! 👋  How can I help you today?


--- Full Conversation ---
User: hi
Assistant: Hello! 👋  How can I help you today?


--- Full Conversation ---
User: hi
Assistant: Hello! 👋  How can I help you today?


--- Full Conversation ---
Assistant: GenAI, or Generative Artificial Intelligence, is a type of artificial intelligence that focuses on creating new content. 

Think of it like this:

* **Traditional AI** is good at analyzing existing data and making predictions. For example, it can analyze your past spending habits to predict what you might buy next.
* **GenAI** goes a step further. It learns patterns and structures from existing data and uses that knowledge to generate brand new content. This could be anything from text and code to images, music, and even videos.

Here are some key things to know about GenAI:

* **How it works:** GenAI models are trained on massive datasets of text, code, images, or o

In [22]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq
llm=ChatGroq(model="gemma2-9b-it")


class State(TypedDict):
    start:str
    messages:Annotated[list,add_messages]



def chatbot(state:State):
    while True:
    
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        state["messages"]=llm.invoke(user_input)
        print(f'{user_input}\n')
        print(state['messages'][-1].content)
    return {"messages":state["messages"]}

def display(state: State):
    """
    Displays the entire conversation history.
    """
    print("\n--- Full Conversation ---")
    for message in state["messages"]:
        print(message)
    return {'messages':state['messages']} # Return state to continue execution

graph_builder=StateGraph(State)



# ✅ Build the AI Graph
graph_builder.add_node("chatbot", chatbot)  # AI response node
graph_builder.add_node("display", display)  # Display node

# ✅ Define Graph Execution Order
graph_builder.add_edge(START, "chatbot")  # Start -> chatbot
graph_builder.add_edge("chatbot", "display")  # chatbot -> display
graph_builder.add_edge("display", END)  # display -> End

# ✅ Compile the Graph
graph = graph_builder.compile()

graph.invoke({'start':'start'})

hi



TypeError: 'AIMessage' object is not subscriptable